## Requirements

In [2]:
!pip install torch==2.0.0+cu118 --index-url https://download.pytorch.org/whl/cu118 -q
!pip install pyg-lib==0.3.1 torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cu118.html -q
!pip install 'torch-geometric==2.4.0' -q

## Code

https://github.com/ki-ljl/PyG-GCN/tree/main -- Semi-Supervised Classification with Graph Convolutional Networks, ICLR 2017

In [3]:
DEVICE = 'cuda'

CLASSES = [
    'RESIDENTIAL',
    'BUSINESS',
    'RECREATION',
    'SPECIAL',
    'INDUSTRIAL',
    'AGRICULTURE',
    'TRANSPORT',
]

NUM_CLASSES = len(CLASSES)

### Data

Создание набора данных из кварталов и графа

In [4]:
import geopandas as gpd
import networkx as nx
import torch
from torch_geometric.data import Data
from torch_geometric.utils.convert import from_networkx
from sklearn.model_selection import train_test_split

blocks_gdf = gpd.read_parquet('blocks.parquet')
adj_graph = nx.read_graphml('adj_graph.graphml')

def get_masks(y, test_size=0.1):

    labels = y.numpy()

    # Разбиваем данные с учетом распределения классов
    train_indices, test_indices = train_test_split(
        range(len(labels)),
        test_size=test_size,
        stratify=labels,  # Это обеспечит сохранение пропорций классов
    )

    # Создаем маски для обучающих и тестовых данных
    train_mask = torch.zeros(len(labels), dtype=torch.bool)
    test_mask = torch.zeros(len(labels), dtype=torch.bool)

    train_mask[train_indices] = True
    test_mask[test_indices] = True

    return train_mask, test_mask


def load_data(blocks_gdf, adj_graph):

    blocks_gdf = blocks_gdf.copy()

    blocks_gdf['length'] = blocks_gdf.length.div(blocks_gdf.length.max())

    blocks_gdf['mrr_area'] = blocks_gdf.geometry.minimum_rotated_rectangle().area
    blocks_gdf['mbc_area'] = blocks_gdf.geometry.minimum_rotated_rectangle().area

    # normalization
    # max_area = blocks_gdf[['area', 'mrr_area', 'mbc_area']].max().max()
    # blocks_gdf['area'] = blocks_gdf['area'].div(max_area)
    # blocks_gdf['mrr_area'] = blocks_gdf['mrr_area'].div(max_area)
    # blocks_gdf['mbc_area'] = blocks_gdf['mbc_area'].div(max_area)

    x = torch.Tensor([[
        row['aspect_ratio'],
        row['length'],
        row['area'],
        row['mbc_area'],
        row['mrr_area'],
    ] for _,row in blocks_gdf.iterrows()])
    y = torch.Tensor([-1 if lu is None else CLASSES.index(lu) for lu in blocks_gdf['land_use']]).long()

    edge_index = from_networkx(adj_graph).edge_index
    edge_attr = from_networkx(adj_graph)['border_length']
    data = Data(x = x, y=y, edge_index=edge_index, edge_attr=edge_attr)
    
    train_mask, test_mask = get_masks(data.y)

    # Добавляем маски в объект data
    data.train_mask = train_mask
    data.test_mask = test_mask
    
    return data

data = load_data(blocks_gdf, adj_graph)

Создание дополнительных признаков

In [5]:
from sklearn.preprocessing import PolynomialFeatures

def add_polynomial_features(data, degree=2):
    poly = PolynomialFeatures(degree, interaction_only=True, include_bias=False)
    new_x = poly.fit_transform(data.x.numpy())  # sklearn работает с numpy
    data.x = torch.tensor(new_x, dtype=torch.float)
    return data

data = add_polynomial_features(data)
print("Размерность после полиномиальных признаков:", data.x.shape)
num_node_features = data.x.shape[1]

Размерность после полиномиальных признаков: torch.Size([16320, 15])


Нормализация

In [6]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

def normalize_features(data, method="standard"):
    scaler = StandardScaler() if method == "standard" else MinMaxScaler()
    data.x = torch.tensor(scaler.fit_transform(data.x.numpy()), dtype=torch.float)
    return data

data = normalize_features(data)
print("Признаки после нормализации:", data.x[:5])

Признаки после нормализации: tensor([[-3.1783e-01,  3.5658e+00,  1.9568e+00,  2.9349e+00,  2.9349e+00,
          8.9094e-04,  1.8419e+00,  2.0613e+00,  2.0613e+00,  8.5312e-01,
          1.2301e+00,  1.2301e+00,  2.6984e-01,  2.6984e-01,  4.1084e-01],
        [ 3.0976e-01,  1.9885e-01, -1.6334e-01, -1.8378e-01, -1.8378e-01,
          4.5974e-02,  1.7040e-01, -2.0611e-02, -2.0611e-02, -8.3562e-02,
         -8.8229e-02, -8.8229e-02, -4.6430e-02, -4.6430e-02, -4.8272e-02],
        [-3.6823e-01,  1.2006e+00,  7.5883e-01,  4.3979e-01,  4.3979e-01,
         -6.5287e-02,  2.1183e-01,  1.9241e-02,  1.9241e-02,  1.0067e-01,
          3.2195e-02,  3.2195e-02, -1.6600e-02, -1.6600e-02, -2.8143e-02],
        [-2.9761e-01,  1.0917e+00,  3.0676e-01,  4.2631e-01,  4.2631e-01,
         -4.1322e-02,  3.5106e-01,  3.4116e-01,  3.4116e-01,  7.6207e-03,
          2.2762e-02,  2.2762e-02, -3.0670e-02, -3.0670e-02, -2.8957e-02],
        [-3.6342e-01,  3.3453e+00,  4.6088e+00,  4.4368e+00,  4.4368e+00,
     

Переводим данные на куду

In [7]:
data = data.to(DEVICE)

In [8]:
import pandas as pd

train_y = list(data.y[data.train_mask].cpu())
test_y = list(data.y[data.test_mask].cpu())

y_df = pd.DataFrame.from_dict({
    'train': [train_y.count(-1), *[train_y.count(i) for i,_ in enumerate(CLASSES)]],
    'test': [test_y.count(-1), *[test_y.count(i) for i,_ in enumerate(CLASSES)]],
}, orient='columns')
y_df.index = ['None', *CLASSES]
y_df

,train,test
None,163,18
RESIDENTIAL,4967,552
BUSINESS,263,29
RECREATION,1704,189
SPECIAL,64,7
INDUSTRIAL,451,50
AGRICULTURE,112,13
TRANSPORT,6964,774


### Training

In [15]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from tqdm import tqdm

class GCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, 32)
        self.norm = torch.nn.BatchNorm1d(32)
        self.conv2 = GCNConv(32, num_classes)

    def forward(self, data):
        x = data.x
        edge_index = data.edge_index
        edge_attr = data.edge_attr

        # Применяем первый слой свертки, учитывая веса ребер
        x = self.conv1(x, edge_index, edge_weight=edge_attr)
        x = self.norm(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_weight=edge_attr)

        return x

def train(model, data, device):
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4) # lp=0.01, weight_decay=1e-4 #3e-4

    class_counts = torch.bincount(data.y[data.y != -1])
    class_weights = 1. / class_counts.float()

    loss_function = torch.nn.CrossEntropyLoss(weight=class_weights, ignore_index=-1).to(device)
    model.train()
    
    pbar = tqdm(range(10_000))
    for _ in pbar:
        out = model(data)
        optimizer.zero_grad()
        loss = loss_function(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        pbar.set_description(f'loss : {round(loss.item(),2)}')
        optimizer.step()

        # print(f'Epoch {epoch:03d} loss {loss.item():.4f}')

device = torch.device(DEVICE)
model = GCN(num_node_features, NUM_CLASSES).to(device)
train(model, data, device)

loss : 1.49: 100%|██████████| 10000/10000 [01:06<00:00, 151.11it/s]


### Test

In [16]:
# from __future__ import division

import torch
import torch.nn.functional as F
from torch_scatter import scatter_add


def accuracy(pred, target):
    return (pred == target).sum().item() / target.numel()

def true_positive(pred, target, num_classes):
    out = []
    for i in range(num_classes):
        out.append(((pred == i) & (target == i)).sum())

    return torch.tensor(out)

def true_negative(pred, target, num_classes):
    out = []
    for i in range(num_classes):
        out.append(((pred != i) & (target != i)).sum())

    return torch.tensor(out)

def false_positive(pred, target, num_classes):
    out = []
    for i in range(num_classes):
        out.append(((pred == i) & (target != i)).sum())

    return torch.tensor(out)

def false_negative(pred, target, num_classes):
    out = []
    for i in range(num_classes):
        out.append(((pred != i) & (target == i)).sum())

    return torch.tensor(out)

def precision(pred, target, num_classes):
    tp = true_positive(pred, target, num_classes).to(torch.float)
    fp = false_positive(pred, target, num_classes).to(torch.float)

    out = tp / (tp + fp)
    out[torch.isnan(out)] = 0

    return out

def recall(pred, target, num_classes):
    tp = true_positive(pred, target, num_classes).to(torch.float)
    fn = false_negative(pred, target, num_classes).to(torch.float)

    out = tp / (tp + fn)
    out[torch.isnan(out)] = 0

    return out

def f1_score(pred, target, num_classes):
    prec = precision(pred, target, num_classes)
    rec = recall(pred, target, num_classes)

    score = 2 * (prec * rec) / (prec + rec)
    score[torch.isnan(score)] = 0

    return score



def mean_iou(pred, target, num_classes, batch=None):
    pred, target = F.one_hot(pred, num_classes), F.one_hot(target, num_classes)

    if batch is not None:
        i = scatter_add(pred & target, batch, dim=0).to(torch.float)
        u = scatter_add(pred | target, batch, dim=0).to(torch.float)
    else:
        i = (pred & target).sum(dim=0).to(torch.float)
        u = (pred | target).sum(dim=0).to(torch.float)

    iou = i / u
    iou[torch.isnan(iou)] = 1
    iou = iou.mean(dim=-1)
    return iou

def test(model, data):
    model.eval()
    _, pred = model(data).max(dim=1)

    mask = (data.test_mask) & (data.y != -1)

    pred = pred[mask]
    y = data.y[mask]

    import pandas as pd

    df = pd.DataFrame.from_dict(data={
        'precision': precision(pred, y, NUM_CLASSES),
        'recall': recall(pred, y, NUM_CLASSES),
        'f1 score': f1_score(pred, y, NUM_CLASSES)
    }, orient='columns').astype(float)
    df.index = [CLASSES[i] for i in df.index]

    return df

test(model, data)

,precision,recall,f1 score
RESIDENTIAL,0.722045,0.409420,0.522543
BUSINESS,0.035874,0.275862,0.063492
RECREATION,0.385965,0.116402,0.178862
SPECIAL,0.028571,0.285714,0.051948
INDUSTRIAL,0.120000,0.240000,0.160000
AGRICULTURE,0.080460,0.538462,0.140000
TRANSPORT,0.780105,0.770026,0.775033


In [17]:
def test(model, data):
    model.eval()
    _, pred = model(data).max(dim=1)

    mask = (data.test_mask) & (data.y != -1)

    correct = int(pred[mask].eq(data.y[mask]).sum().item())
    acc = correct / int(mask.sum())
    print('GCN Accuracy: {:.4f}'.format(acc))
    
test(model, data)

GCN Accuracy: 0.5409


### Output

In [18]:
import torch.nn.functional as F

def get_class_probabilities(model, data):
    # Прогон модели
    model.eval()  # Убедитесь, что модель в режиме оценки
    with torch.no_grad():
        output = model(data)  # Выход модели (логиты)

    # Применяем softmax к выходу модели, чтобы получить вероятности для всех классов
    probabilities = F.softmax(output, dim=1)  # softmax по оси классов (второй размер)

    return [{CLASSES[i]:float(p) for i,p in enumerate(probability)} for probability in probabilities]

get_class_probabilities(model, data)

[{'RESIDENTIAL': 0.0039932867512106895,
  'BUSINESS': 0.000205251868464984,
  'RECREATION': 0.10361684858798981,
  'SPECIAL': 0.004959160927683115,
  'INDUSTRIAL': 0.22189930081367493,
  'AGRICULTURE': 0.6581908464431763,
  'TRANSPORT': 0.0071353428065776825},
 {'RESIDENTIAL': 0.16188953816890717,
  'BUSINESS': 0.07969959080219269,
  'RECREATION': 0.26973333954811096,
  'SPECIAL': 0.01685352995991707,
  'INDUSTRIAL': 0.16946667432785034,
  'AGRICULTURE': 0.006526111625134945,
  'TRANSPORT': 0.29583123326301575},
 {'RESIDENTIAL': 0.041469015181064606,
  'BUSINESS': 0.04131721705198288,
  'RECREATION': 0.1281197965145111,
  'SPECIAL': 0.294587105512619,
  'INDUSTRIAL': 0.16520720720291138,
  'AGRICULTURE': 0.3287363052368164,
  'TRANSPORT': 0.0005633832770399749},
 {'RESIDENTIAL': 0.0618646964430809,
  'BUSINESS': 0.015106556937098503,
  'RECREATION': 0.17568254470825195,
  'SPECIAL': 0.08001623302698135,
  'INDUSTRIAL': 0.5256584882736206,
  'AGRICULTURE': 0.0839938372373581,
  'TRANSPO

In [19]:
blocks_gdf['probabilities'] = get_class_probabilities(model, data)

In [20]:
blocks_gdf.to_parquet('blocks_probabilities.parquet')